# Goal
* 爬取mobile01行動通訊綜合討論區中第一頁主題名稱包含"台灣之星"的文字資料以利文字探勘
* 爬取 標題 回復數量 作者 發文日期 爬取日期 
* 存取為json格式
## Can be improve
* 進入hyperlink爬取討論內容
* 設定成每日爬取 運用AirFlow及資料庫(例如MySQL)
##  We can use the data for......
* 輿論監測 - 該討論串對該主題討論為正面或是負面 ?
* 熱度描述 - A事件發布後，在討論版中被討論的數量站全部主題的 ? % vs 時間戳記

In [236]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime,timezone,timedelta

In [242]:
# header and pages
my_headers = {'user-agent': 'my-app/0.0.1'}
top3_pages = ['https://www.mobile01.com/topiclist.php?f=18',
             'https://www.mobile01.com/topiclist.php?f=18&p=2',
             'https://www.mobile01.com/topiclist.php?f=18&p=3']

In [318]:
result_list = []
keyword_list=['台灣之星','台星']
for (p,link) in enumerate(top3_pages):
    # get request
    res = requests.get(link,
                  headers=my_headers)
    res.encoding='utf-8'
    
    if res.status_code == 200:
        print('accessed successfully', 'crawling...')
    else:
        print('accessed failed......')
        print(res.text)
    # complie by bs4
    soup = BeautifulSoup(res.text)
    # get all td tag
    td_set = [ele for ele in soup.find_all('td')]
    # assign now time with 8+
    tzutc_8 = timezone(timedelta(hours=8))
    now_time = dt.datetime.now(tz=tzutc_8)

    for i,tag in enumerate(td_set):
        result_single = {}
    # filter
        # for each keyword
        for keyword in keyword_list:
        # class - subject    
            if (tag.attrs['class'] == ['subject']) and (keyword in tag.a.string):
    # get subject and authur tag
                result_single['title'] = tag.a.string
                popularity_raw_data = tag.a.attrs['title']
                result_single['popularity'] = [int(s) for s in popularity_raw_data.split() if s.isdigit()][0]
                result_single['replys'] = td_set[ i + 1 ].string
                gene = td_set[ i + 2 ].strings
                result_single['created_timestamp'] = str(next(gene))
                result_single['crawled'] = str(now_time)[:16]
                result_single['author'] = next(gene)
                result_single['page'] = p+1
                result_list.append(result_single)
    # combine result

    result_all ={
        k : [d.get(k) for d in result_list]
        for k in set().union(*result_list)
    }

accessed successfully crawling...
accessed successfully crawling...
accessed successfully crawling...


In [319]:
# to explore
# print(td_set[92])
# print(td_set[92].name)
# print(td_set[92].a)
# print(td_set[92].a.attrs['title'])
# cont = td_set[92].a.attrs['title']
# [int(s) for s in cont.split() if s.isdigit()][0]

# print(td_set[94])
# print(td_set[94].attrs)
# print(td_set[94].find_all('p'))
# gene = td_set[94].strings
# print(next(gene))
# print(next(gene))

In [320]:
with open("result.json","w") as f:
    json.dump(result_all,f)
    print("compelete saving")


compelete saving


In [321]:
with open("result.json",'r') as load_f:
    load_dict = json.load(load_f)
print(load_dict)    

{'author': ['worgooo', 'Nanako0625', 'maybearthur', 'chkilin', 'guiyi', 'a2369627', 'bertram02', 'hank-smith', 'hank-smith', 'harry35'], 'crawled': ['2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28', '2019-03-23 17:28'], 'page': [1, 2, 2, 2, 2, 2, 2, 3, 3, 3], 'title': ['台灣之星', '台灣之星499但門號不限速吃到飽 資訊月再啟動', '台星99-3M真的感覺慢(用慣21M)~~~', '【好友省序號】~台灣之星網路申辦門號~賺300元通話金', '台灣之星188 帳單費用增加', '收到台星99升138的問題', '台星在怕5G開台被三大排擠出市場啦!!!', '最近台灣之星188的上傳速度怎麼怪怪的⋯', '今天台星VIP門號的限定', '遠傳是不是有稍微把台星放在眼裡了？'], 'replys': ['6', '77', '33', '576', '22', '18', '36', '51', '6', '28'], 'created_timestamp': ['2019-03-22 17:55', '2018-11-28 19:07', '2019-03-21 14:13', '2018-03-27 13:44', '2019-03-20 14:29', '2018-09-13 09:43', '2019-03-20 15:19', '2019-03-19 21:33', '2019-03-20 15:59', '2019-03-18 18:52'], 'popularity': [2739, 108847, 14256, 215479, 9075, 7260, 13519, 11352, 2332, 9922]}
